In [1]:
from pennylane import numpy as np
import pennylane as qml

In [23]:
n_bits = 2

dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def decompose_two_qubit_QFT(basis_id, with_swap = False):
    """A circuit that computes the QFT on two qubits using elementary gates.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    if type(basis_id) == type(0):
        bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    elif type(basis_id) == type('0'):
        bits = [int(x) for x in basis_id]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    ##################
    # YOUR CODE HERE #
    ##################
    
    qml.Hadamard(wires = 0)
        
    qml.ctrl(qml.S, control=1)(wires=0)

    qml.Hadamard(wires = 1)

    #swaps the inner rows
    if with_swap:
        qml.QubitUnitary(np.array([[1,0,0,0],
                                   [0,0,1,0],
                                   [0,1,0,0],
                                   [0,0,0,1]]),
                         wires=range(dev.num_wires)
                        )
                    
    return qml.state()


@qml.qnode(dev)
def two_qubit_QFT(basis_id):
    """A circuit that computes the QFT on two qubits using qml.QubitUnitary. 
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
    
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    
    # Prepare the basis state |basis_id>
    if type(basis_id) == type(0):
        bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    elif type(basis_id) == type('0'):
        bits = [int(x) for x in basis_id]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    ##################
    # YOUR CODE HERE #
    ##################
    N = 2**n_bits
    omega = np.exp(2.0*np.pi*1j/N)

    U = np.zeros(shape = (N,N), dtype=complex)
    for i in range(N):
        for j in range(N):
            U[i][j] = omega**(i * j)
    
    U = U / np.sqrt(N)
    
    qml.QubitUnitary(U, wires=range(n_bits))
    return qml.state()


In [29]:
for ss in ['00','01','10','11']:
    print(ss)
    print(np.around(decompose_two_qubit_QFT(ss, True),8))
    print(np.around(two_qubit_QFT(ss),8))

00
[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
01
[ 0.5+0.j   0. +0.5j -0.5+0.j  -0. -0.5j]
[ 0.5+0.j   0. +0.5j -0.5+0.j  -0. -0.5j]
10
[ 0.5+0.j -0.5+0.j  0.5+0.j -0.5+0.j]
[ 0.5+0.j -0.5+0.j  0.5-0.j -0.5+0.j]
11
[ 0.5+0.j  -0. -0.5j -0.5+0.j   0. +0.5j]
[ 0.5+0.j  -0. -0.5j -0.5+0.j   0. +0.5j]


In [5]:
decompose_two_qubit_QFT(0)

tensor([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j], requires_grad=True)

In [6]:
decompose_two_qubit_QFT('01')

tensor([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j], requires_grad=True)

In [7]:
decompose_two_qubit_QFT('10')

tensor([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j], requires_grad=True)

In [13]:
decompose_two_qubit_QFT('11')

tensor([ 5.000000e-01+0.j , -3.061617e-17-0.5j, -5.000000e-01+0.j ,
         3.061617e-17+0.5j], requires_grad=True)

In [14]:
decompose_two_qubit_QFT('11', True)

tensor([ 5.000000e-01+0.j , -5.000000e-01+0.j , -3.061617e-17-0.5j,
         3.061617e-17+0.5j], requires_grad=True)

In [79]:
def three_qubit_QFT(num_wires):
    #act on qubit 0
    print(f'hadamard 0')
    k=2
    omega = 2*np.pi/2**k
    print(f'rot {k}: omega {omega}, control by 1, act on 0')
    
    k=3
    omega = 2*np.pi/2**k
    print(f'rot {k}: omega {omega}, control by 2, act on 0')
    
    #act on qubit 1
    print('hadamard 1')
    k = 2
    omega = 2*np.pi/2**k
    print(f'rot {k}: omega {omega}, control by 2, act on 1')
    
    #act on qubit 2
    print('hadamard 2')

    #swap
    print('swap 0 <-> 2')

In [76]:
def n_qubit_QFT(num_wires):
    """A circuit that computes the QFT on three qubits.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
        
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    #bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    #qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    
    ##################
    # YOUR CODE HERE #
    ################## 

    for wire in range(1,num_wires+1):
        #qml.Hadamard(wires=wire)
        print(f'hadmard: {wire - 1}')
        for i, w in enumerate(range(wire+1, num_wires+1)):
            k = i + 2
            #print(f'k:{k}')
            omega = 2*np.pi/2**k
            print(f'rot {k}: omega {omega}, control by {w - 1}, act on {wire - 1}')
            #qml.ctrl(qml.RZ(2*np.pi*1j/(2**k)), control=wire+k)
            
    for w in range(num_wires//2):
        print(f'swap {w} <-> {num_wires-w-1}')
        
        
#     for wire in range(1, dev.num_wires+1):
#         qml.Hadamard(wires=wire-1)
#         for i, w in enumerate(range(wire+1, dev.num_wires+1)):
#             k = i+2
#             qml.CRZ(2*np.pi*1j/(2**k), wires=[w-1, wire-1])
#             #qml.ctrl(qml.RZ(2*np.pi*1j/(2**k)), control=w-1)(wires=wire)
            
#     for wire in range(dev.num_wires//2):
#         qml.SWAP(wires=[wire,dev.num_wires-wire-1])
        
#     return qml.state()

# for wire in range(1, dev.num_wires+1):
#         qml.Hadamard(wires=wire-1)
#         for i, w in enumerate(range(wire+1, dev.num_wires+1)):
#             k = i+2
#             qml.CRZ(2*np.pi*1j/(2**k), wires=[w-1, wire-1])
#             #qml.ctrl(qml.RZ(2*np.pi*1j/(2**k)), control=w-1)(wires=wire)
            
#     for wire in range(dev.num_wires//2):
#         qml.SWAP(wires=[wire,dev.num_wires-wire-1])
        
#     return qml.state()

In [80]:
three_qubit_QFT(3)

hadamard 0
rot 2: omega 1.5707963267948966, control by 1, act on 0
rot 3: omega 0.7853981633974483, control by 2, act on 0
hadamard 1
rot 2: omega 1.5707963267948966, control by 2, act on 1
hadamard 2
swap 0 <-> 2


In [78]:
n_qubit_QFT(3)

hadmard: 0
rot 2: omega 1.5707963267948966, control by 1, act on 0
rot 3: omega 0.7853981633974483, control by 2, act on 0
hadmard: 1
rot 2: omega 1.5707963267948966, control by 2, act on 1
hadmard: 2
swap 0 <-> 2


In [74]:
n_qubit_QFT(4)

hadmard: 0
rot 2: omega 1.5707963267948966j, control by 1, act on 0
rot 3: omega 0.7853981633974483j, control by 2, act on 0
rot 4: omega 0.39269908169872414j, control by 3, act on 0
hadmard: 1
rot 2: omega 1.5707963267948966j, control by 2, act on 1
rot 3: omega 0.7853981633974483j, control by 3, act on 1
hadmard: 2
rot 2: omega 1.5707963267948966j, control by 3, act on 2
hadmard: 3
swap 0 <-> 3
swap 1 <-> 2


In [75]:
n_qubit_QFT(5)

hadmard: 0
rot 2: omega 1.5707963267948966j, control by 1, act on 0
rot 3: omega 0.7853981633974483j, control by 2, act on 0
rot 4: omega 0.39269908169872414j, control by 3, act on 0
rot 5: omega 0.19634954084936207j, control by 4, act on 0
hadmard: 1
rot 2: omega 1.5707963267948966j, control by 2, act on 1
rot 3: omega 0.7853981633974483j, control by 3, act on 1
rot 4: omega 0.39269908169872414j, control by 4, act on 1
hadmard: 2
rot 2: omega 1.5707963267948966j, control by 3, act on 2
rot 3: omega 0.7853981633974483j, control by 4, act on 2
hadmard: 3
rot 2: omega 1.5707963267948966j, control by 4, act on 3
hadmard: 4
swap 0 <-> 4
swap 1 <-> 3


In [81]:
def build_fft(n_bits):
    dev = qml.device("default.qubit", wires=n_bits)

    @qml.qnode(dev)
    def fft_circuit(basis_id):
        if type(basis_id) == type(0):
            bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
        elif type(basis_id) == type('0'):
            bits = [int(x) for x in basis_id]
        
        qml.BasisStatePreparation(bits, wires=range(dev.num_wires))
        qml.QFT(wires=range(dev.num_wires))
        return qml.state()
    
    return fft_circuit

In [86]:
build_fft(3)('000')

tensor([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
        0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j], requires_grad=True)

In [101]:
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def three_qubit_QFT(basis_id):
    """A circuit that computes the QFT on three qubits.
    
    Args:
        basis_id (int): An integer value identifying the basis state to construct.
        
    Returns:
        array[complex]: The state of the qubits after the QFT operation.
    """
    # Prepare the basis state |basis_id>
    if type(basis_id) == type(0):
        bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
    elif type(basis_id) == type('0'):
        bits = [int(x) for x in basis_id]
            
    qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    
    ##################
    # YOUR CODE HERE #
    ################## 
    def omega(k):
        return 2.0*np.pi/(2**k)
        
    qml.Hadamard(0)
    qml.ctrl(qml.PhaseShift, control=1)(omega(2), wires=0)
    qml.ctrl(qml.PhaseShift, control=2)(omega(3), wires=0)
    #qml.CRZ(omega(2), wires=[1,0])
    #qml.CRZ(omega(3), wires=[2,0])
    
    qml.Hadamard(1)
    qml.ctrl(qml.PhaseShift, control=2)(omega(2), wires=1)

    #qml.CRZ(omega(2), wires=[2,1])
    
    qml.Hadamard(2)
    
    qml.SWAP(wires=[0,2])
    return qml.state()

In [102]:
three_qubit_QFT(0)

tensor([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
        0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j], requires_grad=True)

In [103]:
for i in range(2**3):
    print(i)
    print(np.around(build_fft(3)(i), 8))
    print(np.around(three_qubit_QFT(i), 8))
    

0
[0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j
 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j]
[0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j
 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j 0.35355339+0.j]
1
[ 0.35355339+0.j          0.25      +0.25j        0.        +0.35355339j
 -0.25      +0.25j       -0.35355339+0.j         -0.25      -0.25j
 -0.        -0.35355339j  0.25      -0.25j      ]
[ 0.35355339+0.j          0.25      +0.25j        0.        +0.35355339j
 -0.25      +0.25j       -0.35355339+0.j         -0.25      -0.25j
 -0.        -0.35355339j  0.25      -0.25j      ]
2
[ 0.35355339+0.j          0.        +0.35355339j -0.35355339+0.j
 -0.        -0.35355339j  0.35355339-0.j          0.        +0.35355339j
 -0.35355339+0.j         -0.        -0.35355339j]
[ 0.35355339+0.j          0.        +0.35355339j -0.35355339+0.j
 -0.        -0.35355339j  0.35355339+0.j          0.        +0.35355339j
 -0.35355339+0.j         -0.        -0.353

In [107]:
def build_n_qubit_fft(n_bits):
    
    def omega(k):
        return 2.0*np.pi/(2**k)
    
    dev = qml.device("default.qubit", wires=n_bits)
    @qml.qnode(dev)
    def fft_circuit(basis_id):
        if type(basis_id) == type(0):
            bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
        elif type(basis_id) == type('0'):
            bits = [int(x) for x in basis_id]
        
        qml.BasisStatePreparation(bits, wires=range(dev.num_wires))
        
        for wire in range(1, n_bits+1):
            qml.Hadamard(wires=wire-1)
            for i, w in enumerate(range(wire+1, n_bits+1)):
                k = i+2
                qml.ctrl(qml.PhaseShift, control=w-1)(omega(k), wires=wire-1)
            
        for w in range(n_bits//2):
            qml.SWAP(wires=[w,n_bits-w-1])
        
        return qml.state()
    
    return fft_circuit

In [110]:
nn = 4
for i in range(2**nn):
    print(i)
    print(np.around(build_fft(nn)(i), 8))
    print(np.around(build_n_qubit_fft(nn)(i), 8))
    

0
[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
[0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j
 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.25+0.j]
1
[ 0.25      +0.j          0.23096988+0.09567086j  0.1767767 +0.1767767j
  0.09567086+0.23096988j -0.        +0.25j       -0.09567086+0.23096988j
 -0.1767767 +0.1767767j  -0.23096988+0.09567086j -0.25      -0.j
 -0.23096988-0.09567086j -0.1767767 -0.1767767j  -0.09567086-0.23096988j
  0.        -0.25j        0.09567086-0.23096988j  0.1767767 -0.1767767j
  0.23096988-0.09567086j]
[ 0.25      +0.j          0.23096988+0.09567086j  0.1767767 +0.1767767j
  0.09567086+0.23096988j  0.        +0.25j       -0.09567086+0.23096988j
 -0.1767767 +0.1767767j  -0.23096988+0.09567086j -0.25      +0.j
 -0.23096988-0.09567086j -0.1767767 -0.1767767j  -0.09567086-0.23096988j
 -0.        -0.25j        0.09567086-0

# Recursive QFT

In [119]:
def build_recursive_n_qubit_fft(n_bits):
    
    def omega(k):
        return 2.0*np.pi/(2**k)
    
    def qft_recursive_rotations(n_qubits, wire=0):
        """A circuit that performs the QFT rotations on the specified qubits
            recursively.

        Args:
            n_qubits (int): An integer value identifying the number of qubits.
            wire (int): An integer identifying the wire 
                        (or the qubit) to apply rotations on.
        """

        ##################
        # YOUR CODE HERE #
        ################## 
        print(f'H wires {wire}')
        qml.Hadamard(wires=wire)
        for i, w in enumerate(range(wire+1, n_qubits)):
            k = i+2
            print(f'i:{i}, w:{w}, k:{k}, rot({omega(k)}) control {w} on {wire}')
            qml.ctrl(qml.PhaseShift, control=w)(omega(k), wires=wire)
            
        if wire + 1 < n_qubits:
            qft_recursive_rotations(n_qubits, wire + 1)


    dev = qml.device("default.qubit", wires=n_bits)
    @qml.qnode(dev)
    def fft_circuit(basis_id):
        if type(basis_id) == type(0):
            bits = [int(x) for x in np.binary_repr(basis_id, width=dev.num_wires)]
        elif type(basis_id) == type('0'):
            bits = [int(x) for x in basis_id]
        
        qml.BasisStatePreparation(bits, wires=range(dev.num_wires))
        
        qft_recursive_rotations(n_bits)
            
        for w in range(n_bits//2):
            qml.SWAP(wires=[w,n_bits-w-1])
        
        return qml.state()
    
    return fft_circuit

In [122]:
np.around(build_recursive_n_qubit_fft(4)(1), 8)

H wires 0
i:0, w:1, k:2, rot(1.5707963267948966) control 1 on 0
i:1, w:2, k:3, rot(0.7853981633974483) control 2 on 0
i:2, w:3, k:4, rot(0.39269908169872414) control 3 on 0
H wires 1
i:0, w:2, k:2, rot(1.5707963267948966) control 2 on 1
i:1, w:3, k:3, rot(0.7853981633974483) control 3 on 1
H wires 2
i:0, w:3, k:2, rot(1.5707963267948966) control 3 on 2
H wires 3


tensor([ 0.25      +0.j        ,  0.23096988+0.09567086j,
         0.1767767 +0.1767767j ,  0.09567086+0.23096988j,
         0.        +0.25j      , -0.09567086+0.23096988j,
        -0.1767767 +0.1767767j , -0.23096988+0.09567086j,
        -0.25      +0.j        , -0.23096988-0.09567086j,
        -0.1767767 -0.1767767j , -0.09567086-0.23096988j,
        -0.        -0.25j      ,  0.09567086-0.23096988j,
         0.1767767 -0.1767767j ,  0.23096988-0.09567086j], requires_grad=True)

In [124]:
np.around(build_fft(4)(1), 8)

tensor([ 0.25      +0.j        ,  0.23096988+0.09567086j,
         0.1767767 +0.1767767j ,  0.09567086+0.23096988j,
        -0.        +0.25j      , -0.09567086+0.23096988j,
        -0.1767767 +0.1767767j , -0.23096988+0.09567086j,
        -0.25      -0.j        , -0.23096988-0.09567086j,
        -0.1767767 -0.1767767j , -0.09567086-0.23096988j,
         0.        -0.25j      ,  0.09567086-0.23096988j,
         0.1767767 -0.1767767j ,  0.23096988-0.09567086j], requires_grad=True)

In [ ]:
nn = 4
for i in range(2**nn):
    print(i)
    print(np.around(build_fft(nn)(i), 8))
    print(np.around(build_n_qubit_fft(nn)(i), 8))
    